In [162]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from math import sin, cos, pi
import imageio
import os

%matplotlib inline

def pend(y, t, b, c):
    "Second order pendulum ODE."
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt


def generate_frames(img_dir,
                    rot0=0, 
                    theta0=[np.pi/8, 0.0],
                    rot_speed = 0.1, rot_accel=0,delta_t=2,
                    steps = 501, n_imgs = 200,
                    trail=50):
    """Generate an image for each frame.
    
    Each frame lasts delta_t seconds.
    """
    count_speed = 0
    count_accel = 0 

    x_tot = []
    y_tot = []

    for jj in range(n_imgs):

        t = np.linspace(jj*delta_t, (jj+1)*delta_t, steps)
        t = t[1:]

        dt = delta_t/steps

        # solution to 2nd order pendulum ODE
        sol = odeint(pend, theta0, t, args=(b, c))

        # projection of the pendulum on (x,y) plane
        sin_L = np.asarray([sin(theta) for theta in (sol[:,0])])

        x = []
        y = []
        for sinL in (sin_L):

            rot = np.radians(rot0+count_speed*rot_speed*dt)
            cs, sn = np.cos(rot), np.sin(rot)
            R = np.array(((cs,-sn), (sn, cs)))

            xy = np.zeros(2)
            xy[0] = L*sinL
            xy[1] = 0

            # rotation by rot radians of (x,y) coordinate
            xy_prime = np.matmul(R,xy)
            x.append(xy_prime[0])
            y.append(xy_prime[1])

            x_tot.append(xy_prime[0])
            y_tot.append(xy_prime[1])

            count_speed+=1

        rot_speed = rot_speed + rot_accel*count_accel*dt
        count_accel +=1

        
        # pop off the end of the total solution, i.e create a trail effect
        if len(x_tot) > trail*len(sin_L):
            x_tot = x_tot[len(sin_L):]
            y_tot = y_tot[len(sin_L):]
        
        theta0 = sol[-1]
        fig = plt.figure(1,figsize=(6,6)) 
        ax = fig.add_subplot(111, aspect='equal')

        ax.plot(x_tot,y_tot,color='r')
        ax.plot(x,y,color='r') #head color

        ax.set_xlim((-L/2,L/2))
        ax.set_ylim((-L/2,L/2))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.savefig( img_dir + "/" + '{:04}'.format(jj) +'.png')
        plt.clf()
        

def make_gif(imgs_dir, n_imgs=50, fps=1/30):
    "Make a gif from the generated frames"
    filenames = os.listdir(img_dir)
    filenames.sort()

    filenames = filenames[0:n_imgs]
    with imageio.get_writer(img_dir + '/movie.gif', mode='I',duration=fps) as writer:
        for filename in filenames:
            if ".png" in filename:
                image = imageio.imread(img_dir + '/' + filename)
                writer.append_data(image)

        filenames = filenames[-1:0:-1]

        for filename in filenames:
            if ".png" in filename:

                image = imageio.imread(img_dir + '/' + filename)
                writer.append_data(image)


In [173]:
# pendulum parameters
b = 0 # friction
L = 10 # Length of string
g = 9.8
c = 5#g/L

rot0 = 0
theta0 = [np.pi/8, 0.0] # initial position of pendulum
rot_speed = 0.5 * 5001 / 100 * 100   # speed of rotation, rad/s
rot_accel = 0#.005
delta_t = 0.1 # time elapsed, in seconds, for a frame
steps = 501 # how many steps in a given delta_t, i.e. resolution
trail = 30 # defines how long the tail of the snake will be

n_imgs = 200
img_dir = 'images6'
fps=1/15

generate_frames(img_dir, rot0, theta0, rot_speed, rot_accel, delta_t, steps , n_imgs, trail)
make_gif(img_dir, n_imgs, fps=fps)
